In [1]:
# Импорты
import asyncio
import asyncpg
import pandas as pd
import json
import os
from datetime import datetime
from pathlib import Path

print("📦 Импорты готовы")

📦 Импорты готовы


In [8]:
# Конфигурация подключения к БД
DB_CONFIG = {
    "host": "localhost",
    "port": 5432,
    "database": "news_analyzer",
    "user": "staspalatov",
    "password": "2004стас",
}

# Настройки экспорта для Kaggle
OUTPUT_DIR = "./kaggle_dataset"
MAX_SAMPLES = 10000

os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"📁 Папка для экспорта: {OUTPUT_DIR}")

📁 Папка для экспорта: ./kaggle_dataset


In [12]:
# Загрузка данных из PostgreSQL
async def load_training_data():
    """Загружает данные для обучения из PostgreSQL"""

    try:
        print("🔌 Подключение к PostgreSQL...")
        conn = await asyncpg.connect(**DB_CONFIG)

        query = """
        SELECT 
            m.text,
            a.hashtags,
            a.sentiment,
            m.channel_title
        FROM messages m
        JOIN analyses a ON m.message_id = a.message_id
        WHERE 
            LENGTH(m.text) BETWEEN 50 AND 2000
            AND a.hashtags IS NOT NULL
            AND a.hashtags != '[]'
            AND a.hashtags != 'null'
            AND jsonb_array_length(a.hashtags::jsonb) BETWEEN 2 AND 10
        ---    AND m.text NOT LIKE '%@%'
        ---    AND m.text NOT LIKE '%http%'
        ORDER BY m.date DESC
        LIMIT $1
        """

        print(f"📊 Загружаем данные (макс. {MAX_SAMPLES})...")
        rows = await conn.fetch(query, MAX_SAMPLES)
        await conn.close()

        print(f"✅ Загружено {len(rows)} записей")
        return rows

    except Exception as e:
        print(f"❌ Ошибка подключения: {e}")
        print("💡 Убедитесь, что PostgreSQL запущен и доступен")
        return []


# Загружаем данные
raw_data = await load_training_data()

🔌 Подключение к PostgreSQL...
📊 Загружаем данные (макс. 10000)...
✅ Загружено 127 записей


In [14]:
# Обработка и форматирование данных
def prepare_training_examples(raw_data):
    """Подготавливает примеры в формате для Saiga LLaMA3"""

    training_examples = []
    system_prompt = "Ты эксперт по генерации хештегов для новостей. Сгенерируй 3-5 релевантных хештегов на русском языке."

    for row in raw_data:
        try:
            hashtags = json.loads(row["hashtags"])
            hashtags_str = ", ".join(hashtags)

            # Формат ChatML для Saiga LLaMA3
            full_prompt = f"<s>system\n{system_prompt}</s>\n<s>user\n{row['text']}</s>\n<s>assistant\n{hashtags_str}</s>"

            training_examples.append(
                {
                    "text": full_prompt,
                    "input": row["text"],
                    "output": hashtags_str,
                    "channel": row["channel_title"],
                    "sentiment": row["sentiment"],
                }
            )

        except Exception as e:
            print(f"⚠️ Ошибка обработки записи: {e}")
            continue

    print(f"✅ Подготовлено {len(training_examples)} примеров")
    return training_examples


# Подготавливаем примеры
if raw_data:
    training_examples = prepare_training_examples(raw_data)
else:
    print("❌ Нет данных для обработки")
    training_examples = []

✅ Подготовлено 127 примеров


In [15]:
# Сохранение данных для Kaggle
def save_for_kaggle(training_examples):
    """Сохраняет данные в формате для загрузки в Kaggle"""

    if not training_examples:
        print("❌ Нет данных для сохранения")
        return

    # Разделение на train/test
    split_idx = int(len(training_examples) * 0.8)
    train_data = training_examples[:split_idx]
    test_data = training_examples[split_idx:]

    print(f"📊 Разделение: {len(train_data)} train, {len(test_data)} test")

    # Сохранение JSONL файлов
    train_file = Path(OUTPUT_DIR) / "train_data.jsonl"
    test_file = Path(OUTPUT_DIR) / "test_data.jsonl"

    with open(train_file, "w", encoding="utf-8") as f:
        for item in train_data:
            f.write(json.dumps(item, ensure_ascii=False) + "\n")

    with open(test_file, "w", encoding="utf-8") as f:
        for item in test_data:
            f.write(json.dumps(item, ensure_ascii=False) + "\n")

    # Конфигурация для Kaggle
    config = {
        "dataset_info": {
            "total_samples": len(training_examples),
            "train_samples": len(train_data),
            "test_samples": len(test_data),
            "created_at": datetime.now().isoformat(),
        },
        "model_config": {
            "base_model": "IlyaGusev/saiga_llama3_8b",
            "task": "hashtag_generation",
            "format": "ChatML",
        },
        "training_config": {
            "epochs": 3,
            "batch_size": 4,
            "learning_rate": 2e-4,
            "lora_r": 16,
            "lora_alpha": 32,
        },
    }

    config_file = Path(OUTPUT_DIR) / "config.json"
    with open(config_file, "w", encoding="utf-8") as f:
        json.dump(config, f, ensure_ascii=False, indent=2)

    # README для Kaggle
    readme_content = f"""# Датасет для дообучения Saiga LLaMA3

## 📊 Информация
- Всего записей: {len(training_examples)}
- Обучающих: {len(train_data)}
- Тестовых: {len(test_data)}

## 📁 Файлы
- `train_data.jsonl` - обучающие данные
- `test_data.jsonl` - тестовые данные  
- `config.json` - конфигурация обучения

## 🚀 Использование в Kaggle
Загрузите эти файлы как dataset в Kaggle и используйте notebook для обучения.
"""

    readme_file = Path(OUTPUT_DIR) / "README.md"
    with open(readme_file, "w", encoding="utf-8") as f:
        f.write(readme_content)

    print(f"\n🎉 Данные сохранены в {OUTPUT_DIR}/")
    print(f"📄 Файлы:")
    print(f"   - train_data.jsonl ({len(train_data)} записей)")
    print(f"   - test_data.jsonl ({len(test_data)} записей)")
    print(f"   - config.json")
    print(f"   - README.md")

    return {
        "train_samples": len(train_data),
        "test_samples": len(test_data),
        "total_samples": len(training_examples),
    }


# Сохраняем данные
result = save_for_kaggle(training_examples)

📊 Разделение: 101 train, 26 test

🎉 Данные сохранены в ./kaggle_dataset/
📄 Файлы:
   - train_data.jsonl (101 записей)
   - test_data.jsonl (26 записей)
   - config.json
   - README.md
